In [9]:
# using GuSTO"
using JLD2, FileIO
include("../src/GuSTO.jl")

Forest

In [10]:
function setUpEnvironmentSphereObstacles()
    env = ISSCorner()
    add_obstacles!(env)
    return env
end

setUpEnvironmentSphereObstacles (generic function with 1 method)

In [11]:
function setUpProblemAllParams(N, r_init, r_goal, q_init, q_goal, robot, model, env, tf_guess)
    goal_set = GoalSet()
    addGoals!(goal_set, r_init, r_goal, q_init, q_goal, tf_guess)
    x_init = [r_init; zeros(3); q_init; zeros(3)]
    PD = ProblemDefinition(robot, model, env, x_init, goal_set);
    TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
    TOSgusto = TrajectoryOptimizationSolution(TOP)
    return TOP, TOSgusto
end

setUpProblemAllParams (generic function with 1 method)

In [12]:
function addGoals!(goal_set, r_init, r_goal, q_init, q_goal, tf_guess)
    q_goal = q_goal/norm(q_goal)
    x_init = [r_init; zeros(3); q_init; zeros(3)]
    v_goal = zeros(3)
    ω_goal = zeros(3)

    x_goal = [r_goal; v_goal; q_goal; ω_goal]
    # add_goal!(goal_set, Goal(PointGoal(x_goal), tf_guess, model))

    ε = 1e-4
    add_goal!(goal_set, Goal(PointGoal(r_goal), tf_guess, 1:3))
    add_goal!(goal_set, Goal(PointGoal(v_goal), tf_guess, 4:6))
    add_goal!(goal_set, Goal(BoxGoal(q_goal.-ε, q_goal.+ε), tf_guess, 7:10))
    # add_goal!(goal_set, Goal(PointGoal(q_goal), tf_guess, 7:10))
    add_goal!(goal_set, Goal(PointGoal(ω_goal), tf_guess, 11:13))
end

addGoals! (generic function with 1 method)

In [13]:
function createTrainingInputAndOutput(traj_X_vec, traj_U_vec, ω_vec, Δ_vec, dual_vec = nothing)
    X_row = hcat((map(a -> a[:], traj_X_vec))...)'
    U_row = hcat((map(a -> a[:], traj_U_vec))...)'
    if dual_vec != nothing
        dual_row = hcat((map(a -> a[:], dual_vec))...)'
    end
#     @show(size(ω_vec))
#     @show(size(X_row))
    if dual_vec == nothing
        train_input = hcat(X_row, U_row, ω_vec, Δ_vec)[1:end-1,:]
    else
        train_input = hcat(X_row, U_row, ω_vec, Δ_vec, dual_row)[1:end-1,:]
    end

    diff_X = traj_X_vec[2:end]-traj_X_vec[1:end-1]
    diff_U = traj_U_vec[2:end]-traj_U_vec[1:end-1]
    diff_X_row = hcat((map(a -> a[:], diff_X))...)'
    diff_U_row = hcat((map(a -> a[:], diff_U))...)'

    if dual_vec == nothing
        train_output = hcat(diff_X_row, diff_U_row)
    else
        train_output = hcat(diff_X_row, diff_U_row, dual_row[2:end, :])
    end
    
    if size(train_input,1) != size(train_output,1)
        println("ERROR: train input and output have different # of rows")
    end
    if size(train_input,2) - 2 != size(train_output,2)
        println("ERROR: each row of train input should 2 elements more than output")
    end
    
    return train_input, train_output
    
end

createTrainingInputAndOutput (generic function with 2 methods)

In [14]:
function checkIfInISS(r, env)
    # Must be INSIDE any keep in
    # Must be OUTSIDE all keep out - ignoring this one for now
    # Must be OUTSIDE all obstacles
    inside_keepin = false
    for z in env.keepin_zones
        maxVals = z.origin + z.widths
        minVals = z.origin - z.widths
        if (all(r.<maxVals) && all(r.>minVals))
            inside_keepin = true
            break # Inside ANY is pass
        end
    end
    if !inside_keepin
        return false
    end
    
#     inside_keepout = false
#     for z in env.keepout_zones
#         maxVals = z.origin + z.widths
#         minVals = z.origin - z.widths
#         if (all(r.<maxVals) && all(r.>minVals))
#             inside_keepout = true
#             break # If inside ANY then we fail
#         end
#     end
#     if inside_keepout
#         println("Is inside some keep out zones")
#         return false
#     end
    
    for o in env.obstacle_set
        dist_from_center = norm(r - o.center)
        if dist_from_center < o.r
            return false
        end
    end
    
    return true
end

checkIfInISS (generic function with 1 method)

# Only along one corridor

In [15]:
env = setUpEnvironmentSphereObstacles();
N = 50
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
tf_guess = 300.

# Pick the init and goal positions
# Small corner maneuver
r_goal = [10.9; 3.0; 5.0]
q_goal = [0.; 1.; 0.; 0.]
v_goal = zeros(3)
ω_goal = zeros(3)
x_goal = [r_goal; v_goal; q_goal; ω_goal]

# r_init = [9.2; -0.1; 5.6] # This works. Keep y at -0.1 and vary x from ?? up to 11.2
# r_init = [8.5; -0.1; 5.6] # z is too high! Doesn't work
# r_init = [8.5; -0.1; 4.6]

x_vals = range(8.5, stop = 8.7, length = 2)
x_vals = range(3.0, stop = 8.7, length = 2)

y_vals = [-0.1]
z_vals = range(4.6, stop = 5.6, length =3)


max_iter = 10

# Input is now (13*50) + (6*50) + 2 + 13 = 965 
# Output is now (13*50) + (6*50) + 13 = 963
train_input_collected_first_only = Array{Float64}(undef,0,965)
train_output_collected_first_only = Array{Float64}(undef,0,963)

train_input_collected_last_only = Array{Float64}(undef,0,965)
train_output_collected_last_only = Array{Float64}(undef,0,963)

train_input_collected = Array{Float64}(undef,0,965)
train_output_collected = Array{Float64}(undef,0,963)

notInISSCount = 0
problemNumber = 0
alongEachDim = 100
totalCount = 0
errorCount = 0

for x in x_vals
    for y in y_vals
        println("-------")
        println("-------")
        println("Completed $(totalCount*100.0/(length(x_vals)*length(y_vals)*length(z_vals))) %")
        for z in z_vals
            totalCount = totalCount + 1

            r = [x, y, z]
            # r = [9.2; -0.1; 5.6]
            if !checkIfInISS(r, env)                
                notInISSCount = notInISSCount + 1
                continue
            end
            r_init = r
#             for i in 1:4
                for i = 1
                problemNumber = problemNumber +1
                println("------------------------")
                @show problemNumber
                q_init = zeros(4)
                q_init[i] = 1.
                @show r_init
                @show q_init
                TOP, TOSgusto = setUpProblemAllParams(N, r_init, r_goal, q_init, q_goal, robot, model, env, tf_guess)
                traj_init = init_traj_straightline(TOP)

                traj_X_vec = [traj_init.X] 
                traj_U_vec = [traj_init.U]
                traj_X_vec_first_only = [traj_init.X]
                traj_U_vec_first_only = [traj_init.U]
                traj_X_vec_last_only = [traj_init.X]
                traj_U_vec_last_only = [traj_init.U]

                dual_vec = [zeros(13)]

                # Solve one step at a time with IPOPT
                try
                    solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, traj_init, "Ipopt", max_iter=1, print_level=0);
                catch e
                    println("$e")
                    errorCount = errorCount + 1
                    continue
                end
                push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                push!(traj_X_vec_first_only, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec_first_only, TOSgusto.SCPS.traj.U)
                push!(dual_vec, TOSgusto.SCPS.dual)
                iterCount = 1
                # @show iterCount

                while !TOSgusto.SCPS.converged && iterCount <max_iter && (TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED)
                    solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, TOSgusto.SCPS.traj, "Ipopt", max_iter=1, print_level=0);
                    if TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED 
                        push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                        push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                        push!(dual_vec, TOSgusto.SCPS.dual)
                        iterCount = iterCount + 1
                        # @show iterCount
                    end
                end

                if TOSgusto.SCPS.scp_status[end] == :OK 
                    if TOSgusto.SCPS.converged
                        push!(traj_X_vec_last_only, TOSgusto.SCPS.traj.X)
                        push!(traj_U_vec_last_only, TOSgusto.SCPS.traj.U)
                        println("Saving data solution converged and is OK")               
                        # traj_X_vec is of size num_iter * 13 * 50
                        # traj_U_vec is of size num_iter * 6 * 50
                        # ω_vec is of size num_iter * 1
                        # Δ_vec is of size num_iter * 1
                        # dual_vec is of size num_iter * 13
                        train_input, train_output = createTrainingInputAndOutput(traj_X_vec, traj_U_vec, TOSgusto.SCPS.param.alg.ω_vec, TOSgusto.SCPS.param.alg.Δ_vec, dual_vec)
                        train_input_first_only, train_output_first_only = createTrainingInputAndOutput(traj_X_vec_first_only, traj_U_vec_first_only, TOSgusto.SCPS.param.alg.ω_vec[1:2], TOSgusto.SCPS.param.alg.Δ_vec[1:2], dual_vec[1:2])
                        train_input_last_only, train_output_last_only = createTrainingInputAndOutput(traj_X_vec_last_only, traj_U_vec_last_only, [TOSgusto.SCPS.param.alg.ω_vec[1],TOSgusto.SCPS.param.alg.ω_vec[end]], [TOSgusto.SCPS.param.alg.Δ_vec[1],TOSgusto.SCPS.param.alg.Δ_vec[end]], [dual_vec[1],dual_vec[end]])
                        train_input_collected = vcat(train_input_collected, train_input)
                        train_output_collected = vcat(train_output_collected, train_output)
                        train_input_collected_first_only = vcat(train_input_collected_first_only, train_input_first_only)
                        train_output_collected_first_only = vcat(train_output_collected_first_only, train_output_first_only)
                        train_input_collected_last_only = vcat(train_input_collected_last_only, train_input_last_only)
                        train_output_collected_last_only = vcat(train_output_collected_last_only, train_output_last_only)
                        # Verifying that the correct duals are being stored
                        # @show train_input[:,953:965]
                        # @show train_output[:,951:963]
                        # @show dual_vec
                        @save "Corridor.jld2" train_input_collected train_output_collected
                        @save "CorridorFirstOnly.jld2" train_input_collected_first_only train_output_collected_first_only
                        @save "CorridorLastOnly.jld2" train_input_collected_last_only train_output_collected_last_only
                    else
                        println("solution is ok but DID NOT converge")
                    end
                else
                    @show TOSgusto.SCPS.scp_status
                    println("solution DID NOT end in OK")
                    if TOSgusto.SCPS.converged
                        println("solution converged")
                    else
                        println("solution DID NOT converge")
                    end
                    break # Usually pointless to just change the angle so break out of that loop
                end

                
            end
        end
    end
end
println("")
println("------------------------")
@show problemNumber
@show notInISSCount


-------
-------
Completed 0.0 %
------------------------
problemNumber = 1
r_init = [3.0, -0.1, 4.6]
q_init = [1.0, 0.0, 0.0, 0.0]
(TOSgusto.SCPS).scp_status = Symbol[:NA, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints]
solution DID NOT end in OK
solution DID NOT converge
------------------------
problemNumber = 2
r_init = [3.0, -0.1, 5.1]
q_init = [1.0, 0.0, 0.0, 0.0]
(TOSgusto.SCPS).scp_status = Symbol[:NA, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints]
solution DID NOT end in OK
solution DID NOT converge
------------------------
problemNumber = 3
r_init = [3.0, -0.1, 5.6]
q_init = [1.0, 0.0, 0.0, 0.0]
(TOSgusto.SCPS).scp_status = Symbol[:NA, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints]
solution DID NOT end in OK
solution DID NOT converge
-------
-------
Completed 50.0 %
------------------------
p

0

# Get final trajectories

In [ ]:
env = setUpEnvironmentSphereObstacles();
N = 50
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
tf_guess = 100.

# Pick the init and goal positions
# Small corner maneuver
r_goal = [10.9; 3.0; 5.0]
q_goal = [0.; 1.; 0.; 0.]
v_goal = zeros(3)
ω_goal = zeros(3)
x_goal = [r_goal; v_goal; q_goal; ω_goal]



max_iter = 10

# Input is now (13*50) + (6*50) + 2 + 13 = 965 
# Output is now (13*50) + (6*50) + 13 = 963
train_input_collected_first_only = Array{Float64}(undef,0,965)
train_output_collected_first_only = Array{Float64}(undef,0,963)

train_input_collected_last_only = Array{Float64}(undef,0,965)
train_output_collected_last_only = Array{Float64}(undef,0,963)

train_input_collected = Array{Float64}(undef,0,965)
train_output_collected = Array{Float64}(undef,0,963)

notInISSCount = 0
problemNumber = 0
alongEachDim = 100
totalCount = 0
errorCount = 0

for x in range(env.worldAABBmin[1], stop= env.worldAABBmax[1], length = alongEachDim)
    for y in range(env.worldAABBmin[2], stop= env.worldAABBmax[2], length = alongEachDim)
        println("-------")
        println("-------")
        println("Completed $(totalCount*100.0/1e6) %")
        for z in range(env.worldAABBmin[3], stop= env.worldAABBmax[3], length = alongEachDim)
            totalCount = totalCount + 1
            if totalCount <= 51237
                continue
            end
            r = [x, y, z]
            # r = [9.2; -0.1; 5.6]
            if !checkIfInISS(r, env)                
                notInISSCount = notInISSCount + 1
                continue
            end
            r_init = r
            for i in 1:4
                problemNumber = problemNumber +1
                println("------------------------")
                @show problemNumber
                q_init = zeros(4)
                q_init[i] = 1.
                @show r_init
                @show q_init
                TOP, TOSgusto = setUpProblemAllParams(N, r_init, r_goal, q_init, q_goal, robot, model, env, tf_guess)
                traj_init = init_traj_straightline(TOP)

                traj_X_vec = [traj_init.X] 
                traj_U_vec = [traj_init.U]
                traj_X_vec_first_only = [traj_init.X]
                traj_U_vec_first_only = [traj_init.U]
                traj_X_vec_last_only = [traj_init.X]
                traj_U_vec_last_only = [traj_init.U]

                dual_vec = [zeros(13)]

                # Solve one step at a time with IPOPT
                try
                    solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, traj_init, "Ipopt", max_iter=1, print_level=0);
                catch e
                    println("$e")
                    errorCount = errorCount + 1
                    continue
                end
                push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                push!(traj_X_vec_first_only, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec_first_only, TOSgusto.SCPS.traj.U)
                push!(dual_vec, TOSgusto.SCPS.dual)
                iterCount = 1
                # @show iterCount

                while !TOSgusto.SCPS.converged && iterCount <max_iter && (TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED)
                    solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, TOSgusto.SCPS.traj, "Ipopt", max_iter=1, print_level=0);
                    if TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED 
                        push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                        push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                        push!(dual_vec, TOSgusto.SCPS.dual)
                        iterCount = iterCount + 1
                        # @show iterCount
                    end
                end

                if TOSgusto.SCPS.scp_status[end] == :OK 
                    if TOSgusto.SCPS.converged
                        push!(traj_X_vec_last_only, TOSgusto.SCPS.traj.X)
                        push!(traj_U_vec_last_only, TOSgusto.SCPS.traj.U)
                        println("Saving data solution converged and is OK")               
                        # traj_X_vec is of size num_iter * 13 * 50
                        # traj_U_vec is of size num_iter * 6 * 50
                        # ω_vec is of size num_iter * 1
                        # Δ_vec is of size num_iter * 1
                        # dual_vec is of size num_iter * 13
                        train_input, train_output = createTrainingInputAndOutput(traj_X_vec, traj_U_vec, TOSgusto.SCPS.param.alg.ω_vec, TOSgusto.SCPS.param.alg.Δ_vec, dual_vec)
                        train_input_first_only, train_output_first_only = createTrainingInputAndOutput(traj_X_vec_first_only, traj_U_vec_first_only, TOSgusto.SCPS.param.alg.ω_vec[1:2], TOSgusto.SCPS.param.alg.Δ_vec[1:2], dual_vec[1:2])
                        train_input_last_only, train_output_last_only = createTrainingInputAndOutput(traj_X_vec_last_only, traj_U_vec_last_only, [TOSgusto.SCPS.param.alg.ω_vec[1],TOSgusto.SCPS.param.alg.ω_vec[end]], [TOSgusto.SCPS.param.alg.Δ_vec[1],TOSgusto.SCPS.param.alg.Δ_vec[end]], [dual_vec[1],dual_vec[end]])
                        train_input_collected = vcat(train_input_collected, train_input)
                        train_output_collected = vcat(train_output_collected, train_output)
                        train_input_collected_first_only = vcat(train_input_collected_first_only, train_input_first_only)
                        train_output_collected_first_only = vcat(train_output_collected_first_only, train_output_first_only)
                        train_input_collected_last_only = vcat(train_input_collected_last_only, train_input_last_only)
                        train_output_collected_last_only = vcat(train_output_collected_last_only, train_output_last_only)
                        # Verifying that the correct duals are being stored
                        # @show train_input[:,953:965]
                        # @show train_output[:,951:963]
                        # @show dual_vec
#                         @save "FullISS.jld2" train_input_collected train_output_collected
#                         @save "FullISSFirstOnly.jld2" train_input_collected_first_only train_output_collected_first_only
#                         @save "FullISSLastOnly.jld2" train_input_collected_last_only train_output_collected_last_only
                    else
                        println("solution is ok but DID NOT converge")
                    end
                else
                    @show TOSgusto.SCPS.scp_status
                    println("solution DID NOT end in OK")
                    if TOSgusto.SCPS.converged
                        println("solution converged")
                    else
                        println("solution DID NOT converge")
                    end
                    break # Usually pointless to just change the angle so break out of that loop
                end

                
            end
        end
    end
end
println("")
println("------------------------")
@show problemNumber
@show notInISSCount


-------
-------
Completed 0.0 %
-------
-------
Completed 0.01 %
-------
-------
Completed 0.02 %
-------
-------
Completed 0.03 %
-------
-------
Completed 0.04 %
-------
-------
Completed 0.05 %
-------
-------
Completed 0.06 %
-------
-------
Completed 0.07 %
-------
-------
Completed 0.08 %
-------
-------
Completed 0.09 %
-------
-------
Completed 0.1 %
-------
-------
Completed 0.11 %
-------
-------
Completed 0.12 %
-------
-------
Completed 0.13 %
-------
-------
Completed 0.14 %
-------
-------
Completed 0.15 %
-------
-------
Completed 0.16 %
-------
-------
Completed 0.17 %
-------
-------
Completed 0.18 %
-------
-------
Completed 0.19 %
-------
-------
Completed 0.2 %
-------
-------
Completed 0.21 %
-------
-------
Completed 0.22 %
-------
-------
Completed 0.23 %
-------
-------
Completed 0.24 %
-------
-------
Completed 0.25 %
-------
-------
Completed 0.26 %
-------
-------
Completed 0.27 %
-------
-------
Completed 0.28 %
-------
-------
Completed 0.29 %
-------
-----

Completed 3.11 %
-------
-------
Completed 3.12 %
-------
-------
Completed 3.13 %
-------
-------
Completed 3.14 %
-------
-------
Completed 3.15 %
-------
-------
Completed 3.16 %
-------
-------
Completed 3.17 %
-------
-------
Completed 3.18 %
-------
-------
Completed 3.19 %
-------
-------
Completed 3.2 %
-------
-------
Completed 3.21 %
-------
-------
Completed 3.22 %
-------
-------
Completed 3.23 %
-------
-------
Completed 3.24 %
-------
-------
Completed 3.25 %
-------
-------
Completed 3.26 %
-------
-------
Completed 3.27 %
-------
-------
Completed 3.28 %
-------
-------
Completed 3.29 %
-------
-------
Completed 3.3 %
-------
-------
Completed 3.31 %
-------
-------
Completed 3.32 %
-------
-------
Completed 3.33 %
-------
-------
Completed 3.34 %
-------
-------
Completed 3.35 %
-------
-------
Completed 3.36 %
-------
-------
Completed 3.37 %
-------
-------
Completed 3.38 %
-------
-------
Completed 3.39 %
-------
-------
Completed 3.4 %
-------
-------
Completed 3.4

ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 57
r_init = [-5.91789, -9.10991, 5.17636]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 58
r_init = [-5.91789, -9.10991, 5.17636]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 59
r_init = [-5.91789, -9.10991, 5.17636]
q_init = [0.0, 0.0, 1.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 60
r_init = [-5.91789, -9.10991, 5.17636]
q_init = [0.0, 0.0, 0.0, 1.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 61
r_init = [-5.91789, -9.10991, 5.40305]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 62
r_init = [-5.

ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 105
r_init = [-5.91789, -8.80595, 3.58951]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 106
r_init = [-5.91789, -8.80595, 3.58951]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 107
r_init = [-5.91789, -8.80595, 3.58951]
q_init = [0.0, 0.0, 1.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 108
r_init = [-5.91789, -8.80595, 3.58951]
q_init = [0.0, 0.0, 0.0, 1.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 109
r_init = [-5.91789, -8.80595, 3.8162]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 110
r_init =

ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 154
r_init = [-5.91789, -8.50198, 1.54927]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 155
r_init = [-5.91789, -8.50198, 1.54927]
q_init = [0.0, 0.0, 1.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 156
r_init = [-5.91789, -8.50198, 1.54927]
q_init = [0.0, 0.0, 0.0, 1.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 157
r_init = [-5.91789, -8.50198, 1.77596]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 158
r_init = [-5.91789, -8.50198, 1.77596]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 159
r_init 

ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 202
r_init = [-5.91789, -8.50198, 4.26959]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 203
r_init = [-5.91789, -8.50198, 4.26959]
q_init = [0.0, 0.0, 1.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 204
r_init = [-5.91789, -8.50198, 4.26959]
q_init = [0.0, 0.0, 0.0, 1.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 205
r_init = [-5.91789, -8.50198, 4.49628]
q_init = [1.0, 0.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 206
r_init = [-5.91789, -8.50198, 4.49628]
q_init = [0.0, 1.0, 0.0, 0.0]
ErrorException("Invalid coefficient NaN on variable X[1,49].")
------------------------
problemNumber = 207
r_init 

In [10]:
@load "FullISS.jld2" train_input_collected train_output_collected
@load "FullISSFirstOnly.jld2" train_input_collected_first_only train_output_collected_first_only
@load "FullISSLastOnly.jld2" train_input_collected_last_only train_output_collected_last_only
@show size(train_input_collected)
@show size(train_input_collected_last_only)

size(train_input_collected) = (21, 965)
size(train_input_collected_last_only) = (3, 965)


(3, 965)

In [1]:
@show problemNumber
@show notInISSCount
@show totalCount
@show size(train_input_collected)
@show size(train_input_collected_first_only)
@show size(traj_X_vec_first_only)
# Up to totalCount = 51237, problemNumber = 3420, notInISSCount = 47817
# We got no train_input_collected :( 

UndefVarError: UndefVarError: problemNumber not defined

# Older version changing qs (incorrectly)

In [10]:
env = setUpEnvironmentSphereObstacles()
N = 50
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
tf_guess = 10.

# Pick the init and goal positions
# Small corner maneuver
r_init = [11.2; -0.8; 5.6]
r_goal = [10.9; 3.0; 5.0]
q_init = [1.; 0.; 0.; 0.]
# q_goal = [1.; 0.; 0.; 0.]
# q_goal = [1.; 0.5; 0.5; 0.5]
# q_goal = [1.; 0.2; 0.3; 0.4]

q1Pos = [1.]
q2Pos = [0.1 0.2 0.3 0.4 0.5 0.6]
q3Pos = q2Pos
q4Pos = q2Pos

max_iter = 10

# Input is now (13*50) + (6*50) + 2 + 13 = 965 
# Output is now (13*50) + (6*50) + 13 = 963
train_input_collected_first_only = Array{Float64}(undef,0,965)
train_output_collected_first_only = Array{Float64}(undef,0,963)

train_input_collected = Array{Float64}(undef,0,965)
train_output_collected = Array{Float64}(undef,0,963)

problemNumber = 1
for q1 in q1Pos
    for q2 in q2Pos
        for q3 in q3Pos
            for q4 in q4Pos
                println("")
                @show problemNumber
                q_goal = [q1; q2; q3; q4]
                TOP, TOSgusto = setUpProblemAllParams(N, r_init, r_goal, q_init, q_goal, robot, model, env, tf_guess)
                traj_init = init_traj_straightline(TOP)

                traj_X_vec = [traj_init.X] 
                traj_U_vec = [traj_init.U]
                traj_X_vec_first_only = [traj_init.X]
                traj_U_vec_first_only = [traj_init.U]
                dual_vec = [zeros(13)]

                # Solve one step at a time with IPOPT
                solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, traj_init, "Ipopt", max_iter=1, print_level=0);
                push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                push!(traj_X_vec_first_only, TOSgusto.SCPS.traj.X)
                push!(traj_U_vec_first_only, TOSgusto.SCPS.traj.U)
                push!(dual_vec, TOSgusto.SCPS.dual)
                iterCount = 1

                while !TOSgusto.SCPS.converged && iterCount <max_iter && (TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED)
                    solve_SCP!(TOSgusto, TOP, solve_gusto_jump!, TOSgusto.SCPS.traj, "Ipopt", max_iter=1, print_level=0);
                    if TOSgusto.SCPS.solver_status[end] == MOI.LOCALLY_SOLVED 
                        push!(traj_X_vec, TOSgusto.SCPS.traj.X)
                        push!(traj_U_vec, TOSgusto.SCPS.traj.U)
                        push!(dual_vec, TOSgusto.SCPS.dual)
                        iterCount = iterCount + 1
                        @show iterCount
                    end
                end

                if TOSgusto.SCPS.scp_status[end] == :OK 
                    if TOSgusto.SCPS.converged
                        println("Saving data solution converged and is OK")               
                        # traj_X_vec is of size num_iter * 13 * 50
                        # traj_U_vec is of size num_iter * 6 * 50
                        # ω_vec is of size num_iter * 1
                        # Δ_vec is of size num_iter * 1
                        # dual_vec is of size num_iter * 13
                        train_input, train_output = createTrainingInputAndOutput(traj_X_vec, traj_U_vec, TOSgusto.SCPS.param.alg.ω_vec, TOSgusto.SCPS.param.alg.Δ_vec, dual_vec)
                        train_input_first_only, train_output_first_only = createTrainingInputAndOutput(traj_X_vec_first_only, traj_U_vec_first_only, TOSgusto.SCPS.param.alg.ω_vec[1:2], TOSgusto.SCPS.param.alg.Δ_vec[1:2], dual_vec[1:2])

                        train_input_collected = vcat(train_input_collected, train_input)
                        train_output_collected = vcat(train_output_collected, train_output)
                        train_input_collected_first_only = vcat(train_input_collected_first_only, train_input_first_only)
                        train_output_collected_first_only = vcat(train_output_collected_first_only, train_output_first_only)
                        # Verifying that the correct duals are being stored
                        # @show train_input[:,953:965]
                        # @show train_output[:,951:963]
                        # @show dual_vec
                    else
                        println("solution is ok but DID NOT converge")
                    end
                else
                    @show TOSgusto.SCPS.scp_status
                    println("solution DID NOT end in OK")
                    if TOSgusto.SCPS.converged
                        println("solution converged")
                    else
                        println("solution DID NOT converge")
                    end
                end

                problemNumber = problemNumber + 1
                @save "SphereObstaclesWithDuals.jld2" train_input_collected train_output_collected
                @save "SphereObstaclesOnly1IterWithDuals.jld2" train_input_collected_first_only train_output_collected_first_only
            end
        end
    end
end



problemNumber = 1


UndefVarError: UndefVarError: x_init not defined

In [3]:
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.solver_status
@show TOS_SCP.SCPS.scp_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.ω_vec
@show TOS_SCP.SCPS.param.alg.Δ_vec
@show TOS_SCP.SCPS.J_true
@show TOS_SCP.SCPS.dual
;

(TOS_SCP.SCPS).converged = true
(TOS_SCP.SCPS).iterations = 7
(TOS_SCP.SCPS).total_time = 31.686495691
(TOS_SCP.SCPS).accept_solution = Bool[true, true, true, true, true, true, true, true]
(TOS_SCP.SCPS).solver_status = Union{TerminationStatusCode, Symbol}[:NA, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED]
(TOS_SCP.SCPS).scp_status = Symbol[:NA, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :OK, :OK, :OK, :OK]
(TOS_SCP.SCPS).convergence_measure = [0.0, 0.0688753, 0.00388088, 0.0030732, 0.0110116, 0.000427816, 1.40182e-5, 5.6762e-7]
(((TOS_SCP.SCPS).param).alg).ω_vec = [1.0, 5.0, 25.0, 125.0, 125.0, 125.0, 125.0, 125.0]
(((TOS_SCP.SCPS).param).alg).Δ_vec = [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
(TOS_SCP.SCPS).J_true = Any[0.0, 62.2831, 63.0013, 64.4436, 99.1265, 98.8738, 98.8771, 98.8775]
(TOS_SCP.SCPS).dual = [-1.28574, 655.518, -133.94, -10.2259, 3295.77, -672.769, 0.0001592

In [4]:
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.solver_status
@show TOS_SCP.SCPS.scp_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.ω_vec
@show TOS_SCP.SCPS.param.alg.Δ_vec
@show TOS_SCP.SCPS.J_true
@show TOS_SCP.SCPS.dual
;

(TOS_SCP.SCPS).converged = true
(TOS_SCP.SCPS).iterations = 7
(TOS_SCP.SCPS).total_time = 31.686495691
(TOS_SCP.SCPS).accept_solution = Bool[true, true, true, true, true, true, true, true]
(TOS_SCP.SCPS).solver_status = Union{TerminationStatusCode, Symbol}[:NA, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED, LOCALLY_SOLVED]
(TOS_SCP.SCPS).scp_status = Symbol[:NA, :ViolatesConstraints, :ViolatesConstraints, :ViolatesConstraints, :OK, :OK, :OK, :OK]
(TOS_SCP.SCPS).convergence_measure = [0.0, 0.0688753, 0.00388088, 0.0030732, 0.0110116, 0.000427816, 1.40182e-5, 5.6762e-7]
(((TOS_SCP.SCPS).param).alg).ω_vec = [1.0, 5.0, 25.0, 125.0, 125.0, 125.0, 125.0, 125.0]
(((TOS_SCP.SCPS).param).alg).Δ_vec = [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
(TOS_SCP.SCPS).J_true = Any[0.0, 62.2831, 63.0013, 64.4436, 99.1265, 98.8738, 98.8771, 98.8775]
(TOS_SCP.SCPS).dual = [-1.28574, 655.518, -133.94, -10.2259, 3295.77, -672.769, 0.0001592

In [16]:
using Plots
gr(fmt=:png)
plot()
for i = 7:10
    plot!([collect(1:N)],[TOS_SCP.SCPS.traj.X[i,:]],
        xlabel = "q",
        ylabel = "y",
        legend = :none)
end
plot!()

UndefVarError: UndefVarError: TOS_SCP not defined

In [17]:
gr(fmt=:png)
plot()
plot!([collect(1:N)],[TOS_SCP.SCPS.traj.X[4,:]],
    xlabel = "q",
    ylabel = "y",
    legend = :none)

UndefVarError: UndefVarError: TOS_SCP not defined

In [18]:
gr(fmt=:png)
plot()
plot!([collect(1:N)],[TOS_SCP.SCPS.traj.X[1,:]],
    xlabel = "q",
    ylabel = "y",
    legend = :none)

UndefVarError: UndefVarError: TOS_SCP not defined

In [19]:
using AstrobeeRobot

# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 3

Qs = Vector{Vector{Float64}}()
for k in 1:speed_factor:N
#     q = [quat_inv(TOS_SCP.SCPS.traj.X[7:10,k]); TOS_SCP.SCPS.traj.X[1:3,k]]
    q = [quat_inv(TOS_SCP.SCPS.traj.X[7:10,k]); TOS_SCP.SCPS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)

instantiated a floating joint


UndefVarError: UndefVarError: TOS_SCP not defined